## 1) 앙상블 다:다 모델
앙상블은 여러 모델을 합치는 모델로, 1:1 모델은 없다

In [11]:
#1. 데이터
import numpy as np
x1=np.array([range(1,101), range(101,201)])
y1=np.array([range(1,101), range(101,201)])
x2=np.array([range(501,601), range(601,701)])
y2=np.array([range(501,601), range(601,701)])

x1 = np.transpose(x1)
y1 = np.transpose(y1)
x2 = np.transpose(x2)
y2 = np.transpose(y2)

from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=66, test_size=0.2, shuffle=False)
x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, random_state=66, test_size=0.2, shuffle=False)
x1_val, x1_test, y1_val, y1_test = train_test_split(x1_test, y1_test, random_state=66, test_size=0.5, shuffle=False)
x2_val, x2_test, y2_val, y2_test = train_test_split(x2_test, y2_test, random_state=66, test_size=0.5, shuffle=False)

# 2. 모델 구성
from keras.models import Sequential, Model
from keras.layers import Dense, Input

input1 = Input(shape=(2,))
dense1 = Dense(100, activation='relu')(input1)
dense1 = Dense(30)(dense1)
dense1 = Dense(7)(dense1)

input2 = Input(shape=(2,))
dense2 = Dense(50, activation='relu')(input2)
dense2 = Dense(30)(dense2)
dense2 = Dense(7)(dense2)

# 앙상블
from keras.layers.merge import concatenate
merge1 = concatenate([dense1, dense2])
middle1 = Dense(10)(merge1)
middle2 = Dense(5)(middle1)
middle3 = Dense(30)(middle2)

output1=Dense(30)(middle3)
output1=Dense(7)(output1)
output1=Dense(2)(output1)

output2=Dense(20)(middle3)
output2=Dense(70)(output2)
output2=Dense(2)(output2)

model=Model(inputs=[input1,input2], outputs=[output1,output2])
#model.summary()

# 3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit([x1_train, x2_train],[y1_train, y2_train], epochs=50, batch_size=1, validation_data=([x1_val,x2_val],[y1_val,y2_val]))

# 4. 평가예측
mse = model.evaluate([x1_test,x2_test],[y1_test,y2_test],batch_size=1)
print("MSE : ",mse)
y1_predict, y2_predict = model.predict([x1_test,x2_test])
print("y1 예측값 :", y1_predict, "\n y2 예측값 : ",y2_predict)


Epoch 1/50
80/80 [==============================] - 16s 12ms/step - loss: 104462.4195 - dense_41_loss: 1313.9421 - dense_44_loss: 103148.4821 - dense_41_mse: 1313.9421 - dense_44_mse: 103148.4821 - val_loss: 938.8177 - val_dense_41_loss: 502.9998 - val_dense_44_loss: 435.8179 - val_dense_41_mse: 502.9998 - val_dense_44_mse: 435.8179
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 488.5847 - dense_41_loss: 315.2560 - dense_44_loss: 173.3287 - dense_41_mse: 315.2560 - dense_44_mse: 173.3287 - val_loss: 415.7365 - val_dense_41_loss: 317.1675 - val_dense_44_loss: 98.5690 - val_dense_41_mse: 317.1675 - val_dense_44_mse: 98.5690
Epoch 3/50
80/80 [==============================] - 0s 2ms/step - loss: 319.7418 - dense_41_loss: 234.8444 - dense_44_loss: 84.8974 - dense_41_mse: 234.8444 - dense_44_mse: 84.8974 - val_loss: 1965.3600 - val_dense_41_loss: 1701.1306 - val_dense_44_loss: 264.2291 - val_dense_41_mse: 1701.1306 - val_dense_44_mse: 264.2291
Epoch 4/50
80/80 [====

## 2) 앙상블 다:1 모델

** 가장 많이 나오는 유형임!!!

In [14]:
#1. 데이터
import numpy as np
x1=np.array([range(1,101), range(101,201)])
y=np.array([range(1,101), range(101,201)])
x2=np.array([range(501,601), range(601,701)])

x1=np.transpose(x1)
x2=np.transpose(x2)
y=np.transpose(y)

from sklearn.model_selection import train_test_split
x1_train, x1_test, y_train, y_test = train_test_split(x1, y, random_state=66, test_size=0.2, shuffle=False)
x1_val, x1_test, y_val, y_test = train_test_split(x1_test, y_test, random_state=66, test_size=0.5, shuffle=False)
x2_train, x2_test = train_test_split(x2, random_state=66, shuffle=False, test_size=0.2)
x2_val, x2_test = train_test_split(x2_test, random_state=66, shuffle=False, test_size=0.5)

# 2. 모델 구성
from keras.models import Sequential, Model
from keras.layers import Dense, Input

input1 = Input(shape=(2,))
dense1 = Dense(100, activation='relu')(input1)
dense1 = Dense(30)(dense1)
dense1 = Dense(7)(dense1)

input2 = Input(shape=(2,))
dense2 = Dense(50, activation='relu')(input2)
dense2 = Dense(30)(dense2)
dense2 = Dense(7)(dense2)

from keras.layers.merge import concatenate
merge1=concatenate([dense1,dense2])

output1 = Dense(30)(merge1)
output1 = Dense(7)(output1)
output1 = Dense(2)(output1)

model = Model(inputs=[input1,input2], outputs=output1)

# 3. 훈련
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
model.fit([x1_train, x2_train], y_train, epochs=50, batch_size=1, validation_data=([x1_val, x2_val],y_val))

# 4. 평가예측
mse = model.evaluate([x1_test, x2_test], y_test, batch_size=1)
print("mse :", mse)
y_predict = model.predict([x1_test,x2_test])
print("예측값 \n", y_predict)

Epoch 1/50
80/80 [==============================] - 1s 6ms/step - loss: 2208.7015 - mse: 2208.7015 - val_loss: 414.3950 - val_mse: 414.3950
Epoch 2/50
80/80 [==============================] - 0s 2ms/step - loss: 69.3195 - mse: 69.3195 - val_loss: 25.4426 - val_mse: 25.4426
Epoch 3/50
80/80 [==============================] - 0s 2ms/step - loss: 3.2131 - mse: 3.2131 - val_loss: 3.2274 - val_mse: 3.2274
Epoch 4/50
80/80 [==============================] - 0s 2ms/step - loss: 0.0874 - mse: 0.0874 - val_loss: 2.2537 - val_mse: 2.2537
Epoch 5/50
80/80 [==============================] - 0s 2ms/step - loss: 0.0342 - mse: 0.0342 - val_loss: 1.9934 - val_mse: 1.9934
Epoch 6/50
80/80 [==============================] - 0s 2ms/step - loss: 0.0336 - mse: 0.0336 - val_loss: 1.4590 - val_mse: 1.4590
Epoch 7/50
80/80 [==============================] - 0s 2ms/step - loss: 0.0136 - mse: 0.0136 - val_loss: 1.3340 - val_mse: 1.3340
Epoch 8/50
80/80 [==============================] - 0s 2ms/step - loss: 0.05